# Used Vehicles `SQLite` Database

In [1]:
import os
import random
import warnings
import pandas as pd
import numpy as np
import sqlite3
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Set seed 
seed_value = 42
os.environ['UsedCars_DB'] = str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)

%cd /mnt/UsedCars_Prices/Data

# Read train set
train = pd.read_parquet('usedCars_trainSet.parquet.gzip')
train = train.drop_duplicates()

# Read test set
test = pd.read_parquet('usedCars_testSet.parquet.gzip')
test = test.drop_duplicates()

/mnt/UsedCars_Prices/Data


## Create `SQLite` Database, Examine the Schema and Dimensions

In [2]:
def create_sqlite_database(filename):
    
    ''' Create a SQLite database, connect, create tables, load data, examine schema/rows, close database connection '''
    conn = None
    try:
        conn = sqlite3.connect(filename)
        # Print if a successful connection
        print('Connected to SQLite database')
        cur = conn.cursor()
        # Load train set to database
        cur.execute('''DROP TABLE IF EXISTS TRAIN_SET''')
        train.to_sql('TRAIN_SET', conn, if_exists='replace')
        # Load test set to database        
        cur.execute('''DROP TABLE IF EXISTS TEST_SET''')
        test.to_sql('TEST_SET', conn, if_exists='replace')
        conn.commit()
        print('SQLite Version:', sqlite3.sqlite_version)
    except sqlite3.Error as error:
        print('Failed to add tables', error)
    try:        
        # List the table in the database
        # Getting all tables from sqlite_master
        sql_query = '''SELECT name FROM sqlite_master 
        WHERE type='table';'''
        cur.execute(sql_query)
        print('\nDatabase Tables:')
        print(cur.fetchall())    
    except sqlite3.Error as error:
        print('Failed to execute the above query', error)
        # Examine the schema of the train set 
    try:
        print('\nTrain Set: Schema')
        for row in cur.execute('PRAGMA table_info("TRAIN_SET")').fetchall():
            print(row)
        cur.execute('SELECT * FROM TRAIN_SET') 
        print('\nNumber of Rows: ', len(cur.fetchall()))
        # Examine the schema of the test set         
        print('\nTest Set: Schema')
        for row in cur.execute('PRAGMA table_info("TEST_SET")').fetchall():
            print(row)
        cur.execute('SELECT * FROM TEST_SET') 
        print('\nNumber of Rows: ', len(cur.fetchall()))
    except sqlite3.Error as error:
        print('Failed to execute the above query', error)
    finally:
        if conn:
            conn.close()
            print('\nThe database connection is closed')

if __name__ == '__main__':
    create_sqlite_database('usedcars.db')

Connected to SQLite database
SQLite Version: 3.45.3

Database Tables:
[('TRAIN_SET',), ('TEST_SET',)]

Train Set: Schema
(0, 'index', 'INTEGER', 0, None, 0)
(1, 'price', 'REAL', 0, None, 0)
(2, 'back_legroom', 'REAL', 0, None, 0)
(3, 'body_type', 'TEXT', 0, None, 0)
(4, 'city_fuel_economy', 'REAL', 0, None, 0)
(5, 'daysonmarket', 'REAL', 0, None, 0)
(6, 'engine_displacement', 'REAL', 0, None, 0)
(7, 'front_legroom', 'REAL', 0, None, 0)
(8, 'fuel_tank_volume', 'REAL', 0, None, 0)
(9, 'fuel_type', 'TEXT', 0, None, 0)
(10, 'height', 'REAL', 0, None, 0)
(11, 'highway_fuel_economy', 'REAL', 0, None, 0)
(12, 'is_new', 'INTEGER', 0, None, 0)
(13, 'length', 'REAL', 0, None, 0)
(14, 'listing_color', 'TEXT', 0, None, 0)
(15, 'maximum_seating', 'REAL', 0, None, 0)
(16, 'mileage', 'REAL', 0, None, 0)
(17, 'savings_amount', 'REAL', 0, None, 0)
(18, 'transmission', 'TEXT', 0, None, 0)
(19, 'wheel_system_display', 'TEXT', 0, None, 0)
(20, 'wheelbase', 'REAL', 0, None, 0)
(21, 'width', 'REAL', 0, None

## Exploratory Data Analysis

- Price vs State
- Number per State
- Price per month per state
- Price by Color
- Price by State with Listing Color
- Days on market by State with Listing Color

- Number of vehicles listed per State

In [3]:
# Train set
try:
    conn = sqlite3.connect('usedcars.db')
    cur = conn.cursor()
    sql_query = '''SELECT *, ROUND(100 * freq / SUM(freq) OVER (), 2) AS percentage 
        FROM (
          SELECT State, COUNT(*) AS freq 
          FROM TRAIN_SET 
          GROUP BY State
          ORDER BY freq DESC);'''
    cur.execute(sql_query)
    records = cur.fetchall()
    print('\nTrain Set: Listings Per State')
    headers = [i[0] for i in cur.description]
    print(headers)
    for row in records:
        print(row)
except sqlite3.Error as error:
        print('Failed to execute the above query', error)

# Test set
try:
    conn = sqlite3.connect('usedcars.db')
    cur = conn.cursor()
    sql_query = '''SELECT *, ROUND(100 * freq / SUM(freq) OVER (), 2)  AS percentage  
        FROM (
          SELECT State, COUNT(*) AS freq 
          FROM TEST_SET 
          GROUP BY State
          ORDER BY freq DESC);'''
    cur.execute(sql_query)
    records = cur.fetchall()
    print('\nTest Set: Listings Per State')
    headers = [i[0] for i in cur.description]
    print(headers)
    for row in records:
        print(row)
except sqlite3.Error as error:
        print('Failed to execute the above query', error)
finally:
        if conn:
            conn.close()


Train Set: Listings Per State
['State', 'freq', 'percentage']
('TX', 63624, 24.0)
('CA', 48172, 18.0)
('FL', 47384, 18.0)
('OH', 28897, 11.0)
('IL', 26676, 10.0)
('NY', 23072, 8.0)
('PA', 22859, 8.0)

Test Set: Listings Per State
['State', 'freq', 'percentage']
('TX', 16381, 25.0)
('CA', 11962, 18.0)
('FL', 11691, 17.0)
('OH', 7136, 10.0)
('IL', 6695, 10.0)
('NY', 5816, 8.0)
('PA', 5766, 8.0)


- Price of Vehicle Listings Per State

In [4]:
# Train set
try:
    conn = sqlite3.connect('usedcars.db')
    cur = conn.cursor()
    cur.execute('''SELECT State, COUNT(price) as total_count, MIN(price) AS price_min, ROUND(MAX(price), 2) AS price_max,
        SUM(price) AS price_sum, ROUND(AVG(price), 2) AS price_avg, ROUND(SQRT(AVG(price*price) - AVG(price)*AVG(price)), 2) AS price_stdev, 
        ROUND(AVG(price*price) - AVG(price)*AVG(price), 2) AS price_variance FROM TRAIN_SET GROUP BY State''')
    records = cur.fetchall()
    print('\nTrain Set: Price of Vehicle Listings Per State')
    headers = [i[0] for i in cur.description]
    print(headers)
    for row in records:
        print(row)
except sqlite3.Error as error:
        print('Failed to execute the above query', error)

# Test set
try:
    conn = sqlite3.connect('usedcars.db')
    cur = conn.cursor()
    cur.execute('''SELECT State, COUNT(price) as total_count, MIN(price) AS price_min, ROUND(MAX(price), 2) AS price_max,
        SUM(price) AS price_sum, ROUND(AVG(price), 2) AS price_avg, ROUND(SQRT(AVG(price*price) - AVG(price)*AVG(price)), 2) AS price_stdev, 
        ROUND(AVG(price*price) - AVG(price)*AVG(price), 2) AS price_variance FROM TEST_SET GROUP BY State''')
    records = cur.fetchall()
    print('\nTest Set: Price of Vehicle Listings Per State')
    headers = [i[0] for i in cur.description]
    print(headers)
    for row in records:
        print(row)
except sqlite3.Error as error:
        print('Failed to execute the above query', error)
finally:
        if conn:
            conn.close()


Train Set: Price of Vehicle Listings Per State
['State', 'total_count', 'price_min', 'price_max', 'price_sum', 'price_avg', 'price_stdev', 'price_variance']
('CA', 48172, 6822.0, 50000.0, 1351719453.0, 28060.27, 9416.14, 88663749.56)
('FL', 47384, 5990.0, 50000.0, 1314913025.05, 27750.15, 9734.28, 94756131.28)
('IL', 26676, 7494.0, 50000.0, 747764364.0, 28031.35, 9433.35, 88988036.9)
('NY', 23072, 6994.0, 50000.0, 655787306.55, 28423.51, 9260.06, 85748784.17)
('OH', 28897, 6750.0, 50000.0, 813947512.25, 28167.2, 9613.86, 92426236.43)
('PA', 22859, 6944.0, 50000.0, 645930478.0, 28257.16, 9092.16, 82667393.99)
('TX', 63624, 5999.0, 50000.0, 1846028136.55, 29014.65, 9852.24, 97066625.21)

Test Set: Price of Vehicle Listings Per State
['State', 'total_count', 'price_min', 'price_max', 'price_sum', 'price_avg', 'price_stdev', 'price_variance']
('CA', 11962, 7371.0, 49999.0, 338262903.0, 28278.12, 9482.56, 89918941.73)
('FL', 11691, 4490.0, 50000.0, 323216639.1, 27646.62, 9737.43, 94817497.

- Total Price of Vehicle Listings Per Month Per State

In [5]:
# Train set
try:
    conn = sqlite3.connect('usedcars.db')
    cur = conn.cursor()
    cur.execute('''SELECT State, listed_date_yearMonth, SUM(price) AS price_sum FROM TRAIN_SET GROUP BY listed_date_yearMonth, State''')
    records = cur.fetchall()
    print('\nTrain Set: Total Price of Vehicle Listings Per Month Per State')
    headers = [i[0] for i in cur.description]
    print(headers)
    for row in records:
        print(row)
except sqlite3.Error as error:
        print('Failed to execute the above query', error)

# Test set
try:
    conn = sqlite3.connect('usedcars.db')
    cur = conn.cursor()
    cur.execute('''SELECT State, listed_date_yearMonth, SUM(price) AS price_sum FROM TEST_SET GROUP BY listed_date_yearMonth, State''')
    records = cur.fetchall()
    print('\nTest Set: Total Price of Vehicle Listings Per Month Per State')
    headers = [i[0] for i in cur.description]
    print(headers)
    for row in records:
        print(row)
except sqlite3.Error as error:
        print('Failed to execute the above query', error)
finally:
        if conn:
            conn.close()


Train Set: Total Price of Vehicle Listings Per Month Per State
['State', 'listed_date_yearMonth', 'price_sum']
('CA', '2020-06', 118223072.0)
('FL', '2020-06', 108257979.75)
('IL', '2020-06', 86774411.0)
('NY', '2020-06', 55515085.0)
('OH', '2020-06', 72826675.5)
('PA', '2020-06', 58408660.0)
('TX', '2020-06', 185119095.8)
('CA', '2020-07', 301343697.0)
('FL', '2020-07', 305385015.35)
('IL', '2020-07', 193912692.0)
('NY', '2020-07', 149498355.0)
('OH', '2020-07', 190261974.25)
('PA', '2020-07', 150863788.0)
('TX', '2020-07', 432315248.75)
('CA', '2020-08', 596773404.0)
('FL', '2020-08', 626994487.95)
('IL', '2020-08', 343259631.0)
('NY', '2020-08', 326598957.55)
('OH', '2020-08', 395579213.0)
('PA', '2020-08', 314868051.0)
('TX', '2020-08', 840570198.0)
('CA', '2020-09', 335379280.0)
('FL', '2020-09', 274275542.0)
('IL', '2020-09', 123817630.0)
('NY', '2020-09', 124174909.0)
('OH', '2020-09', 155279649.5)
('PA', '2020-09', 121789979.0)
('TX', '2020-09', 388023594.0)

Test Set: Total P

- Number of Vehicle Listings Per Month Per State

In [6]:
# Train set
try:
    conn = sqlite3.connect('usedcars.db')
    cur = conn.cursor()
    cur.execute('''SELECT State, listed_date_yearMonth, COUNT(*) AS state_count FROM TRAIN_SET GROUP BY listed_date_yearMonth, State''')
    records = cur.fetchall()
    print('\nTrain Set: Number of Vehicle Listings Per Month Per State')
    headers = [i[0] for i in cur.description]
    print(headers)
    for row in records:
        print(row)
except sqlite3.Error as error:
        print('Failed to execute the above query', error)

# Test set
try:
    conn = sqlite3.connect('usedcars.db')
    cur = conn.cursor()
    cur.execute('''SELECT State, listed_date_yearMonth, COUNT(*) AS state_count FROM TEST_SET GROUP BY listed_date_yearMonth, State''')
    records = cur.fetchall()
    print('\nTest Set: Number of Vehicle Listings Per Month Per State')
    headers = [i[0] for i in cur.description]
    print(headers)
    for row in records:
        print(row)
except sqlite3.Error as error:
        print('Failed to execute the above query', error)
finally:
        if conn:
            conn.close()


Train Set: Number of Vehicle Listings Per Month Per State
['State', 'listed_date_yearMonth', 'state_count']
('CA', '2020-06', 4370)
('FL', '2020-06', 3919)
('IL', '2020-06', 3164)
('NY', '2020-06', 1961)
('OH', '2020-06', 2617)
('PA', '2020-06', 2045)
('TX', '2020-06', 6542)
('CA', '2020-07', 10754)
('FL', '2020-07', 10920)
('IL', '2020-07', 6958)
('NY', '2020-07', 5382)
('OH', '2020-07', 6768)
('PA', '2020-07', 5391)
('TX', '2020-07', 14741)
('CA', '2020-08', 21300)
('FL', '2020-08', 22682)
('IL', '2020-08', 12215)
('NY', '2020-08', 11484)
('OH', '2020-08', 14009)
('PA', '2020-08', 11122)
('TX', '2020-08', 28922)
('CA', '2020-09', 11748)
('FL', '2020-09', 9863)
('IL', '2020-09', 4339)
('NY', '2020-09', 4245)
('OH', '2020-09', 5503)
('PA', '2020-09', 4301)
('TX', '2020-09', 13419)

Test Set: Number of Vehicle Listings Per Month Per State
['State', 'listed_date_yearMonth', 'state_count']
('CA', '2020-06', 1078)
('FL', '2020-06', 970)
('IL', '2020-06', 789)
('NY', '2020-06', 520)
('OH',

- Price of Different Colored Vehicles

In [7]:
# Train set
try:
    conn = sqlite3.connect('usedcars.db')
    cur = conn.cursor()
    cur.execute('''SELECT listing_color, COUNT(price) as total_count, MIN(price) AS price_min, ROUND(MAX(price), 2) AS price_max,
        SUM(price) AS price_sum, ROUND(AVG(price), 2) AS price_avg, ROUND(SQRT(AVG(price*price) - AVG(price)*AVG(price)), 2) AS price_stdev, 
        ROUND(AVG(price*price) - AVG(price)*AVG(price), 2) AS price_variance FROM TRAIN_SET GROUP BY listing_color''')
    records = cur.fetchall()
    print('\nTrain Set: Price of Different Colored Vehicles')
    headers = [i[0] for i in cur.description]
    print(headers)
    for row in records:
        print(row)
except sqlite3.Error as error:
        print('Failed to execute the above query', error)

# Test set
try:
    conn = sqlite3.connect('usedcars.db')
    cur = conn.cursor()
    cur.execute('''SELECT listing_color, COUNT(price) as total_count, MIN(price) AS price_min, ROUND(MAX(price), 2) AS price_max,
        SUM(price) AS price_sum, ROUND(AVG(price), 2) AS price_avg, ROUND(SQRT(AVG(price*price) - AVG(price)*AVG(price)), 2) AS price_stdev, 
        ROUND(AVG(price*price) - AVG(price)*AVG(price), 2) AS price_variance FROM TEST_SET GROUP BY listing_color''')
    records = cur.fetchall()
    print('\nTest Set: Price of Different Colored Vehicles')
    headers = [i[0] for i in cur.description]
    print(headers)
    for row in records:
        print(row)
except sqlite3.Error as error:
        print('Failed to execute the above query', error)
finally:
        if conn:
            conn.close()


Train Set: Price of Different Colored Vehicles
['listing_color', 'total_count', 'price_min', 'price_max', 'price_sum', 'price_avg', 'price_stdev', 'price_variance']
('BLACK', 61692, 5990.0, 50000.0, 1787424256.45, 28973.36, 9789.19, 95828313.44)
('BLUE', 27130, 6000.0, 50000.0, 743166760.59, 27392.8, 9241.28, 85401209.54)
('GRAY', 44885, 6458.0, 50000.0, 1245802625.75, 27755.43, 9349.74, 87417658.19)
('RED', 24563, 6000.0, 50000.0, 694765797.75, 28285.05, 9597.0, 92102504.75)
('SILVER', 40119, 6490.0, 50000.0, 1093785101.4, 27263.52, 9397.21, 88307505.04)
('WHITE', 62295, 5999.0, 50000.0, 1811145733.46, 29073.69, 9649.4, 93111005.4)

Test Set: Price of Different Colored Vehicles
['listing_color', 'total_count', 'price_min', 'price_max', 'price_sum', 'price_avg', 'price_stdev', 'price_variance']
('BLACK', 15423, 6205.0, 50000.0, 449246522.25, 29128.35, 9752.16, 95104559.57)
('BLUE', 6889, 6649.0, 50000.0, 188656069.0, 27385.12, 9337.86, 87195600.18)
('GRAY', 11146, 7371.0, 50000.0, 310

- Price of Different Colored Vehicles Per State

In [8]:
# Train set
try:
    conn = sqlite3.connect('usedcars.db')
    cur = conn.cursor()
    cur.execute('''SELECT State, listing_color, COUNT(price) as total_count, MIN(price) AS price_min, ROUND(MAX(price), 2) AS price_max,
        SUM(price) AS price_sum, ROUND(AVG(price), 2) AS price_avg, ROUND(SQRT(AVG(price*price) - AVG(price)*AVG(price)), 2) AS price_stdev, 
        ROUND(AVG(price*price) - AVG(price)*AVG(price), 2) AS price_variance FROM TRAIN_SET GROUP BY State, listing_color''')
    records = cur.fetchall()
    print('\nTrain Set: Price of Different Colored Vehicles Per State')
    headers = [i[0] for i in cur.description]
    print(headers)
    for row in records:
        print(row)
except sqlite3.Error as error:
        print('Failed to execute the above query', error)

# Test set
try:
    conn = sqlite3.connect('usedcars.db')
    cur = conn.cursor()
    cur.execute('''SELECT State, listing_color, COUNT(price) as total_count, MIN(price) AS price_min, ROUND(MAX(price), 2) AS price_max,
        SUM(price) AS price_sum, ROUND(AVG(price), 2) AS price_avg, ROUND(SQRT(AVG(price*price) - AVG(price)*AVG(price)), 2) AS price_stdev, 
        ROUND(AVG(price*price) - AVG(price)*AVG(price), 2) AS price_variance FROM TEST_SET GROUP BY State, listing_color''')
    records = cur.fetchall()
    print('\nTest Set: Price of Different Colored Vehicles Per State')
    headers = [i[0] for i in cur.description]
    print(headers)
    for row in records:
        print(row)
except sqlite3.Error as error:
        print('Failed to execute the above query', error)
finally:
        if conn:
            conn.close()


Train Set: Price of Different Colored Vehicles Per State
['State', 'listing_color', 'total_count', 'price_min', 'price_max', 'price_sum', 'price_avg', 'price_stdev', 'price_variance']
('CA', 'BLACK', 10537, 8488.0, 50000.0, 302696145.0, 28726.98, 9759.56, 95249029.99)
('CA', 'BLUE', 4121, 6822.0, 49999.0, 111456317.0, 27045.94, 9002.33, 81041986.15)
('CA', 'GRAY', 10082, 7975.0, 50000.0, 277811536.0, 27555.2, 9025.25, 81455117.31)
('CA', 'RED', 3022, 7920.0, 49999.0, 81743189.0, 27049.37, 9017.39, 81313257.74)
('CA', 'SILVER', 7700, 7592.0, 49999.0, 206470878.0, 26814.4, 9052.57, 81949002.1)
('CA', 'WHITE', 12710, 7619.0, 50000.0, 371541388.0, 29232.21, 9682.36, 93748114.2)
('FL', 'BLACK', 10264, 5990.0, 50000.0, 292483492.15, 28496.05, 10063.31, 101270293.6)
('FL', 'BLUE', 4984, 6992.0, 50000.0, 134115796.0, 26909.27, 9456.21, 89419934.83)
('FL', 'GRAY', 7991, 6458.0, 49998.0, 220438644.0, 27585.86, 9615.73, 92462193.01)
('FL', 'RED', 4727, 6000.0, 49999.0, 129283105.25, 27349.93, 95

- Number of Days on Market of Different Colored Vehicles Per State

In [9]:
# Train set
try:
    conn = sqlite3.connect('usedcars.db')
    cur = conn.cursor()
    cur.execute('''SELECT State, listing_color, SUM(daysonmarket) AS daysonmarket_sum, ROUND(AVG(daysonmarket), 2) AS daysonmarket_avg, 
    min(daysonmarket) AS daysonmarket_min, MAX(daysonmarket) AS daysonmarket_max FROM TRAIN_SET GROUP BY State, listing_color''')
    records = cur.fetchall()
    print('\nTrain Set:  Number of Days on Market of Different Colored Vehicles Per State')
    headers = [i[0] for i in cur.description]
    print(headers)
    for row in records:
        print(row)
except sqlite3.Error as error:
        print('Failed to execute the above query', error)

# Test set
try:
    conn = sqlite3.connect('usedcars.db')
    cur = conn.cursor()
    cur.execute('''SELECT State, listing_color, SUM(daysonmarket) AS daysonmarket_sum, ROUND(AVG(daysonmarket), 2) AS daysonmarket_avg, 
    min(daysonmarket) AS daysonmarket_min, MAX(daysonmarket) AS daysonmarket_max FROM TEST_SET GROUP BY State, listing_color''')
    records = cur.fetchall()
    print('\nTest Set: Number of Days on Market of Different Colored Vehicles Per State')
    headers = [i[0] for i in cur.description]
    print(headers)
    for row in records:
        print(row)
except sqlite3.Error as error:
        print('Failed to execute the above query', error)
finally:
        if conn:
            conn.close()


Train Set:  Number of Days on Market of Different Colored Vehicles Per State
['State', 'listing_color', 'daysonmarket_sum', 'daysonmarket_avg', 'daysonmarket_min', 'daysonmarket_max']
('CA', 'BLACK', 355517.0, 33.74, 0.0, 108.0)
('CA', 'BLUE', 135880.0, 32.97, 0.0, 103.0)
('CA', 'GRAY', 328177.0, 32.55, 0.0, 105.0)
('CA', 'RED', 102708.0, 33.99, 0.0, 103.0)
('CA', 'SILVER', 261163.0, 33.92, 0.0, 103.0)
('CA', 'WHITE', 395693.0, 31.13, 0.0, 103.0)
('FL', 'BLACK', 322518.0, 31.42, 0.0, 101.0)
('FL', 'BLUE', 153394.0, 30.78, 0.0, 101.0)
('FL', 'GRAY', 248400.0, 31.08, 0.0, 104.0)
('FL', 'RED', 157494.0, 33.32, 0.0, 102.0)
('FL', 'SILVER', 244203.0, 32.99, 0.0, 102.0)
('FL', 'WHITE', 375329.0, 31.24, 0.0, 101.0)
('IL', 'BLACK', 245676.0, 35.21, 0.0, 102.0)
('IL', 'BLUE', 100635.0, 35.69, 0.0, 101.0)
('IL', 'GRAY', 152881.0, 34.79, 0.0, 101.0)
('IL', 'RED', 103285.0, 37.14, 0.0, 102.0)
('IL', 'SILVER', 147546.0, 37.49, 0.0, 102.0)
('IL', 'WHITE', 202377.0, 35.1, 0.0, 101.0)
('NY', 'BLACK',